In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler


In [ ]:
player_data = pd.read_csv("player_data.csv")
opposing_team_data = pd.read_csv("opposing_team_data.csv")

data = pd.merge(player_data, opposing_team_data, left_on="opposing_team", right_on="team_name")
scaler = MinMaxScaler()
data[["PTS", "REB", "AST", "STL",
      "BLK", "TO", "3PM", "off_rtg",
      "def_rtg", "tov_pct", "reb_pct",
      "pace", "pie", "poss", "oreb_pct", 
      "dreb_pct", "efg_pct"]] = scaler.fit_transform(data[["PTS", "REB", "AST", 
                                                           "STL", "BLK", "TO", 
                                                           "3PM", "off_rtg", "def_rtg", 
                                                           "tov_pct", "reb_pct", "pace",
                                                           "pie", "poss", "oreb_pct",
                                                           "dreb_pct", "efg_pct"]])


input_features   = data[["game_id", "player_id", "PTS",
                         "REB", "AST", "STL", "BLK", "TO", 
                         "3PM", "off_rtg", "def_rtg", 
                         "tov_pct","reb_pct", "pace", 
                         "pie", "poss", "oreb_pct", 
                         "dreb_pct", "efg_pct"]]

target_variables = data[["PTS", "REB", "AST", "STL", "BLK", "TO", "3PM"]]
X_train, X_test, y_train, y_test = train_test_split(input_features, target_variables, test_size=0.2)

In [ ]:
weights = np.linspace(1, 0.1, num=40)
weights = np.concatenate((weights, np.zeros(X_train.shape[0] - 40)))

X_train_weighted = X_train * weights
X_test_weighted = X_test * weights
scaler = StandardScaler()
X_train_norm = scaler.fit_transform(X_train_weighted)
X_test_norm = scaler.transform(X_test_weighted)



In [ ]:
input_shape = (40, X_train_weighted.shape[1])
model = tf.keras.Sequential()
model.add(tf.keras.layers.LSTM(32, input_shape=input_shape, use_bias=True))
model.add(tf.keras.layers.Dense(64, activation="relu"))
model.add(tf.keras.layers.Dense(64, activation="relu"))
model.add(tf.keras.layers.Dense(9, activation="linear"))

In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss="mean_squared_error", metrics=["accuracy"])
 

In [ ]:
model.fit(X_train_norm, y_train, epochs=10, batch_size=32, validation_data=(X_test_norm, y_test))

In [ ]:
predictions = model.predict(X_test_norm)
predictions_unnormalized = scaler.inverse_transform(predictions)